In [9]:
import starepandas
import geopandas
import pandas
import datetime

In [2]:
cities = ['Buenos Aires', 'Brasilia', 'Santiago', 
          'Bogota', 'Caracas', 'Sao Paulo', 'Bridgetown']

latitudes = [-34.58, -15.78, -33.45, 4.60, 10.48, -23.55, 13.1]
longitudes = [-58.66, -47.91, -70.66, -74.08, -66.86, -46.63, -59.62]
data =  {'City': cities, 
         'Latitude': latitudes, 'Longitude': longitudes}

cities = starepandas.STAREDataFrame(data)
stare = starepandas.stare_from_xy(cities.Longitude, cities.Latitude, level=27)
geom = geopandas.points_from_xy(cities.Longitude, cities.Latitude, crs='EPSG:4326')
cities.set_stare(stare, inplace=True)
cities.set_geometry(geom, inplace=True)
cities.set_trixels(inplace=True)

In [3]:
countries = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
countries = countries.sort_values(by='name')
countries['geom_simple'] = countries.simplify(0.002)
countries.set_geometry('geom_simple', inplace=True)
samerica = countries[countries.continent=='South America']

stare = starepandas.stare_from_gdf(samerica, level=9, force_ccw=True)
samerica = starepandas.STAREDataFrame(samerica, stare=stare)
trixels = samerica.trixels()
samerica.set_trixels(trixels, inplace=True)

# Geopandas spatial join

In [19]:
geopandas.sjoin(samerica, cities, how='left').head()

,pop_est,continent,name,iso_a3,gdp_md_est,geometry,geom_simple,stare_left,trixels_left,index_right,City,Latitude,Longitude,stare_right,trixels_right
9,44293293,South America,Argentina,ARG,879400.0,"MULTIPOLYGON (((-68.63401 -52.63637, -68.25000...","MULTIPOLYGON (((-68.63401 -52.63637, -68.25000...","[1831909517177126920, 1831927109363171336, 183...",(POLYGON ((-67.84209847159167 -53.768136050714...,0.0,Buenos Aires,-34.58,-58.66,2.663379e+18,POLYGON ((-58.65999966200309 -34.5800001759415...
30,11138234,South America,Bolivia,BOL,78350.0,"POLYGON ((-69.52968 -10.95173, -68.78616 -11.0...","POLYGON ((-69.52968 -10.95173, -68.78616 -11.0...","[2684145377912815620, 2692026677260713989, 270...",(POLYGON ((-62.76427600784291 -18.886669386799...,NaN,NaN,NaN,NaN,NaN,NaN
29,207353391,South America,Brazil,BRA,3081000.0,"POLYGON ((-53.37366 -33.76838, -53.65054 -33.2...","POLYGON ((-53.37366 -33.76838, -53.65054 -33.2...","[2749447572509687812, 2796735368597078020, 286...",(POLYGON ((-53.69933849725345 -4.6308011280983...,1.0,Brasilia,-15.78,-47.91,2.867415e+18,POLYGON ((-47.90999951241486 -15.7799995312697...
29,207353391,South America,Brazil,BRA,3081000.0,"POLYGON ((-53.37366 -33.76838, -53.65054 -33.2...","POLYGON ((-53.37366 -33.76838, -53.65054 -33.2...","[2749447572509687812, 2796735368597078020, 286...",(POLYGON ((-53.69933849725345 -4.6308011280983...,5.0,Sao Paulo,-23.55,-46.63,2.803226e+18,POLYGON ((-46.62999963470963 -23.5500000219939...
10,17789267,South America,Chile,CHL,436100.0,"MULTIPOLYGON (((-68.63401 -52.63637, -68.63335...","MULTIPOLYGON (((-68.63401 -52.63637, -68.63335...","[1829024398665842695, 1831839148432949255, 183...",(POLYGON ((-70.49911208115833 -54.876220492643...,2.0,Santiago,-33.45,-70.66,2.723775e+18,POLYGON ((-70.66000060651589 -33.4499996318796...


In [20]:
starepandas.stare_join(samerica, cities, how='left').head()

,pop_est,continent,name,iso_a3,gdp_md_est,geometry,geom_simple,stare_left,trixels_left,_key_left,City,Latitude,Longitude,stare_right,trixels_right
index_left,,,,,,,,,,,,,,,
9,44293293,South America,Argentina,ARG,879400.0,"MULTIPOLYGON (((-68.63401 -52.63637, -68.25000...","MULTIPOLYGON (((-68.63401 -52.63637, -68.25000...","[1831909517177126920, 1831927109363171336, 183...",(POLYGON ((-67.84209847159167 -53.768136050714...,9.0,Buenos Aires,-34.58,-58.66,2.663379e+18,POLYGON ((-58.65999966200309 -34.5800001759415...
30,11138234,South America,Bolivia,BOL,78350.0,"POLYGON ((-69.52968 -10.95173, -68.78616 -11.0...","POLYGON ((-69.52968 -10.95173, -68.78616 -11.0...","[2684145377912815620, 2692026677260713989, 270...",(POLYGON ((-62.76427600784291 -18.886669386799...,29.0,Brasilia,-15.78,-47.91,2.867415e+18,POLYGON ((-47.90999951241486 -15.7799995312697...
29,207353391,South America,Brazil,BRA,3081000.0,"POLYGON ((-53.37366 -33.76838, -53.65054 -33.2...","POLYGON ((-53.37366 -33.76838, -53.65054 -33.2...","[2749447572509687812, 2796735368597078020, 286...",(POLYGON ((-53.69933849725345 -4.6308011280983...,10.0,Santiago,-33.45,-70.66,2.723775e+18,POLYGON ((-70.66000060651589 -33.4499996318796...
10,17789267,South America,Chile,CHL,436100.0,"MULTIPOLYGON (((-68.63401 -52.63637, -68.63335...","MULTIPOLYGON (((-68.63401 -52.63637, -68.63335...","[1829024398665842695, 1831839148432949255, 183...",(POLYGON ((-70.49911208115833 -54.876220492643...,32.0,Bogota,4.60,-74.08,2.667982e+18,POLYGON ((-74.08000055364545 4.600000473178588...
32,47698524,South America,Colombia,COL,688000.0,"POLYGON ((-66.87633 1.25336, -67.06505 1.13011...","POLYGON ((-66.87633 1.25336, -67.06505 1.13011...","[2666130979403333637, 2667256879310176261, 266...",(POLYGON ((-75.36119345626975 8.42105871140566...,40.0,Caracas,10.48,-66.86,2.494082e+18,POLYGON ((-66.86000048291646 10.48000050718471...


## Hand-rolling a join

In [6]:
left_df = cities
right_df = samerica

left_key = []
right_key = []
for i, row in right_df.iterrows():  
    k = left_df.index[left_df.intersects(row.geometry)]
    left_key.extend(list(k))
    right_key.extend([i]*len(k))

indices = pandas.DataFrame({'_key_left': left_key, 
                            '_key_right':right_key})
indices

,_key_left,_key_right
0,0,9
1,1,29
2,5,29
3,2,10
4,3,32
5,4,40


In [7]:
left_key = []
right_key = []

for i, row in right_df.iterrows():  
    k = left_df.index[left_df.stare_intersects(row.stare)]
    left_key.extend(list(k))
    right_key.extend([i]*len(k))

indices = pandas.DataFrame({'_key_left': left_key, 
                            '_key_right':right_key})

indices

,_key_left,_key_right
0,0,9
1,1,29
2,5,29
3,2,10
4,3,32
5,4,40


In [8]:
# Inner join
joined = left_df
joined = joined.merge(indices, left_index=True, right_index=True)
joined = joined.merge(right_df, left_on='_key_right', right_index=True)
joined = joined.set_index('_key_left')
joined = joined.drop(["_key_right"], axis=1)

In [9]:
# Left Join
index_left = 'index_left'
left_df.index = left_df.index.rename(index_left)
left_df = left_df.reset_index()

joined = left_df
joined = joined.merge(indices, left_index=True, right_index=True, how="left")
joined = joined.merge(right_df.drop(right_df.geometry.name, axis=1),
                      how="left",
                      left_on="_key_right",
                      right_index=True,
                      suffixes=("_left", "_right"))
joined = joined.set_index(index_left)
joined = joined.drop(["_key_right"], axis=1)

# Granule Intersect

In [4]:
fname = 'data/MYD05_L2.A2020060.1635.061.2020061153519.hdf'
modis = starepandas.read_modis_base(fname)
geom = geopandas.points_from_xy(modis.lon, modis.lat, crs='EPSG:4326')
modis.set_geometry(geom, inplace=True)

brazil = samerica[samerica.name=='Brazil']

## STARE

In [10]:
start = datetime.datetime.now()
a = modis.stare_intersects(brazil.stare.iloc[0])
print(datetime.datetime.now() - start)
a

0:01:34.528180


0         0
1         0
2         0
3         0
4         0
         ..
108940    0
108941    0
108942    0
108943    0
108944    0
Length: 108945, dtype: int32

## Conventional

In [11]:
start = datetime.datetime.now()
b = modis.intersects(brazil.iloc[0].geometry)
print(datetime.datetime.now() - start)

0:00:25.555042


In [14]:
modis[a.astype(numpy.bool)]

,lat,lon,stare,geometry
10,-5.206626,-35.363682,2832897501247775625,POINT (-35.36368 -5.20663)
11,-5.230205,-35.513844,2832893603086513833,POINT (-35.51384 -5.23021)
12,-5.253164,-35.660286,2832776718956286153,POINT (-35.66029 -5.25316)
13,-5.275473,-35.802807,2832780306866082601,POINT (-35.80281 -5.27547)
14,-5.297083,-35.941063,2848547756690966825,POINT (-35.94106 -5.29708)
...,...,...,...,...
69363,4.230491,-51.543751,2763963599489179018,POINT (-51.54375 4.23049)
69364,4.218096,-51.626427,2763972369628709674,POINT (-51.62643 4.21810)
69630,4.284687,-51.392056,2763964558042227466,POINT (-51.39206 4.28469)
69631,4.272283,-51.472164,2763965228699433418,POINT (-51.47216 4.27228)
